In [86]:
import numpy as np
import pandas as pd
from bokeh.plotting import figure
from bokeh.io import output_notebook, show
from bokeh.resources import INLINE
output_notebook()

Loading BokehJS ...

In [94]:
from bokeh.layouts import row,column,gridplot,layout
from bokeh.models.widgets import (
    TextInput, Button, Div, DatePicker, Slider, 
    RadioButtonGroup, Select, DataTable, TableColumn,
    NumberFormatter
    )
from bokeh.models import ColumnDataSource, NumeralTickFormatter


In [88]:
def return_calculation(doc):
    
    # create some widgets
    button = Button(label="Submit", button_type="success")
    buy_value = TextInput(title="Buy Value", value='2500')
    buy_date = DatePicker(title='Buy Date', value='2019-01-01')
    sell_value = TextInput(title='Sell Value', value='2620')
    sell_date = DatePicker(title='Sell Date', value='2019-03-25')
    qty = TextInput(title='Quantity', value='100')
    output = Div()

    # add a callback to a widget
    def update():
        bv = float(buy_value.value)
        sv = float(sell_value.value)
        q = int(qty.value)
        start = pd.to_datetime(buy_date.value)
        end = pd.to_datetime(sell_date.value)
        period = (end - start).days
        profit = q*(sv-bv)
        ret = profit/bv
        ann_ret = ret * (365/period)
        text = """
        Net profit = {profit} <br/>
        Holding period = {days} days <br/>
        Return = {ret}% <br/>
        Annualized return = {ann_ret}%
        """
        output.text = text.format(profit=profit, days=period,
                                  ret=round(ret, 2), 
                                  ann_ret=round(ann_ret, 2))
    
    button.on_click(update)

    # create a layout for everything
    l = column(
        row(buy_value, buy_date),
        row(sell_value, sell_date),
        row(qty,button),
        output
    )

    # add the layout to curdoc
    doc.add_root(l)
    
show(return_calculation)

In [90]:
def samples(mu=0.01, sigma=0.01, size=10):
    """
    Return samples from a lognormal distribution
    """
    return np.random.lognormal(mu, sigma, size)    

def get_data(data):
    """
    Data for grid plot
    """
    size = len(data)
    r = np.random.choice(range(size), 5)*0.9
    r = r.astype(int)
    C = np.cumprod # shortcut to cumprod
    df = pd.DataFrame({
        'period': range(size),
        'v1': C(data)
    })
    df.loc[r[0]:, 'v2'] = C(data[r[0]:])
    df.loc[r[1]:, 'v3'] = C(data[r[1]:])
    df.loc[r[2]:, 'v4'] = C(data[r[2]:])
    df.loc[r[3]:, 'v5'] = C(data[r[3]:])
    df.loc[r[4]:, 'v6'] = C(data[r[4]:])
    return df

def get_data_table(data):
    """
    Get data from the dataframe for displaying as datatable
    """
    s = pd.Series({col: data[col].first_valid_index() for col in data})
    df = pd.concat([s, data.iloc[-1]], axis=1)
    df = df.iloc[1:].reset_index()
    df.columns = ['strategy', 'start_period', 'cumulative_returns']
    return df


In [95]:
def drawdown_effect(doc):
    
    # create some widgets
    button = Button(label="Submit", button_type="success")
    returns = Slider(start=-20,end=30,value=2,step=0.25,
                     title='Returns in percentage')
    label = Div(text="Volatility")
    volatility =  RadioButtonGroup(
        labels=["Very Low", "Low", "Medium", "High", "Very High"],
        active=2)
    periods = Slider(start=1, end=200, value=20, step=1,
                    title="Number of periods")  
    src = ColumnDataSource({'period': [], 'v1':[], 'v2': [],
                           'v3': [], 'v4': [], 'v5':[], 'v6':[]})
    src2 = ColumnDataSource({'start_period': [], 'cumulative_returns': []})
    table = DataTable(source=src2, columns=[
        TableColumn(field='start_period', title='start_period'),
        TableColumn(field='cumulative_returns', 
                    title='cumulative_returns',
                   formatter=NumberFormatter(format='$0.00'))
    ], width=250, height=250)
    table_text = """
    Cumulative returns for $1 invested <br>
    in the same strategy <br>
    but in different periods
    """
    table_label = Div(text=table_text)
   
        
    # add a callback to a widget
    def update():
        mu = float(returns.value)*0.01
        sigma = (int(volatility.active)+1)*mu*2
        size = int(periods.value)
        print(mu)
        s = samples(mu=mu, sigma=sigma, size=size)
        data = get_data(s)
        tbl = get_data_table(data)
        src.data = src.from_df(data)
        src2.data = src.from_df(tbl)
            
    figs = []
    for i in range(6):
        figs.append(figure(title='Cumulative Profit for $1 invested',
                          tooltips=[('x', '$x')]))
    for i, col in zip(range(6), ['v1', 'v2', 'v3', 'v4', 'v5', 'v6']):
        figs[i].line('period', col, source=src)
        figs[i].yaxis[0].formatter = NumeralTickFormatter(format="$0.00")
        
    button.on_click(update)
    
    
    grid = gridplot([column(table_label, table)] + 
                    [fig for fig in figs], 
                    ncols=3, plot_width=250, plot_height=250)
    l = layout([
        [returns, periods],
        [label, volatility],
        [button],
        [grid]
        
    ])
    
    # add the layout to curdoc
    doc.add_root(l)
    
show(drawdown_effect)